In [1]:
import pandas as pd

# URL du jeu de données
url = "https://github.com/GU4243-ADS/spring2018-project1-ginnyqg/raw/master/data/spooky.csv"

# Importer le fichier CSV dans un DataFrame
df = pd.read_csv(url)

# Afficher les 10 premiers échantillons
print(df.head(10))

        id                                               text author
0  id26305  This process, however, afforded me no means of...    EAP
1  id17569  It never once occurred to me that the fumbling...    HPL
2  id11008  In his left hand was a gold snuff box, from wh...    EAP
3  id27763  How lovely is spring As we looked from Windsor...    MWS
4  id12958  Finding nothing else, not even gold, the Super...    HPL
5  id22965  A youth passed in solitude, my best years spen...    MWS
6  id09674  The astronomer, perhaps, at this point, took r...    EAP
7  id13515        The surcingle hung in ribands from my body.    EAP
8  id19322  I knew that you could not say to yourself 'ste...    EAP
9  id00912  I confess that neither the structure of langua...    MWS


In [2]:
import re

def reduire_repetitions(texte):
    """
    Cette fonction réduit toute séquence de caractères répétés à 2 occurrences.
    Par exemple : "cooooool" devient "cool".
    """
    # La fonction lambda remplace la séquence capturée par 2 occurrences du caractère
    return re.sub(r'(.)\1+', lambda m: m.group(1) * 2, texte)


In [3]:

# Exemple d'utilisation
texte_initial = "cooooool"
texte_nettoye = reduire_repetitions(texte_initial)
print(texte_nettoye)  # Affiche "cool"

cool


In [19]:
# Dictionnaire de correspondances pour quelques homoglyphes
homoglyphes = {
    # Symboles spéciaux
    '$': 's',
    '@': 'a',
    '0': 'o',
    '1': 'i',  # selon le contexte, vous pouvez choisir 'i'
    '!': 'i',
    '¢': 'c',
    '£': 'l',

    # Lettres grecques
    'Α': 'A',  # Alpha
    'Β': 'B',  # Beta
    'Ε': 'E',  # Epsilon
    'Ζ': 'Z',  # Zeta
    'Η': 'H',  # Eta
    'Ι': 'I',  # Iota
    'Κ': 'K',  # Kappa
    'Μ': 'M',  # Mu
    'Ν': 'N',  # Nu
    'Ο': 'O',  # Omicron
    'Ρ': 'P',  # Rho
    'Τ': 'T',  # Tau
    'Χ': 'X',  # Chi

    # Lettres cyrilliques
    'А': 'A',
    'В': 'B',
    'С': 'C',
    'Е': 'E',
    'Н': 'H',
    'К': 'K',
    'М': 'M',
    'О': 'O',
    'Р': 'P',
    'Т': 'T',
    'Х': 'X'
}

def normaliser_texte(texte):
    """
    Remplace dans le texte les homoglyphes définis dans le dictionnaire par leur équivalent.
    """
    for glyph, remplacement in homoglyphes.items():
        texte = texte.replace(glyph, remplacement)
    return texte

# Exemple d'utilisation
texte_exemple = "$tupide, c00l, l@zy, Αlpha, Вeta"
texte_normalise = normaliser_texte(texte_exemple)

print("Avant normalisation :", texte_exemple)
print("Après normalisation :", texte_normalise)


Avant normalisation : $tupide, c00l, l@zy, Αlpha, Вeta
Après normalisation : stupide, cool, lazy, Alpha, Beta


In [ ]:
import re
import string
import nltk
from nltk.corpus import stopwords

# Assurez-vous d'avoir téléchargé la liste des stopwords pour le français
nltk.download('stopwords')

def preprocess_text(text):
    """
    Applique plusieurs étapes de prétraitement :
      1. Remplacement des URL, adresses e-mail et suppression des balises HTML.
      2. Conversion du texte en minuscules.
      3. Suppression de la ponctuation.
      4. Suppression des mots vides.
    """
    # 1. Remplacer les URL par le token <URL>
    text = re.sub(r'https?://\S+|www\.\S+', 'url', text)
    
    # Remplacer les adresses e-mail par le token <EMAIL>
    text = re.sub(r'\S+@\S+', 'email', text)
    
    # Supprimer les balises HTML (on peut aussi les remplacer par un token si besoin)
    text = re.sub(r'<[^>]+>', ' ', text)
    
    # 2. Mettre tous les caractères en minuscule
    text = text.lower()
    
    # 3. Supprimer la ponctuation
    # On utilise str.translate pour enlever tous les caractères de la chaîne string.punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    # 4. Supprimer les mots vides (stopwords) en français
    stop_words = set(stopwords.words('french'))
    # On tokenize simplement le texte par découpage sur les espaces
    tokens = text.split()
    tokens = [token for token in tokens if token not in stop_words]
    
    # Rejoindre les tokens pour obtenir le texte prétraité
    processed_text = ' '.join(tokens)
    return processed_text

# Exemple d'utilisation
texte_exemple = ("Bonjour, visitez notre site http://exemple.com pour plus d'infos. "
                 "Contactez-nous à info@example.com. Voici un <b>texte en HTML</b> !")
texte_pretraite = preprocess_text(texte_exemple)

print("Texte original :")
print(texte_exemple)
print("\nTexte prétraité :")
print(texte_pretraite)


Détecter les langues présentes dans le dataset et les traduire vers la langue la plus fréquente.

In [20]:
import pandas as pd
from langdetect import detect, DetectorFactory
from googletrans import Translator
from tqdm import tqdm

# Pour rendre la détection de langue reproductible
DetectorFactory.seed = 0


# Supposons que la colonne contenant le texte s'appelle "text".
# Si le nom de la colonne est différent, modifiez-le en conséquence.
# Si besoin, inspectez les colonnes du DataFrame avec : print(df.columns)
if 'text' not in df.columns:
    raise ValueError("La colonne 'text' n'existe pas dans le dataset.")

# Fonction de détection de la langue avec gestion d'exception
def detect_lang(text):
    try:
        return detect(text)
    except Exception as e:
        # En cas d'erreur (texte trop court, vide, etc.), on retourne None
        return None

# Détecter la langue pour chaque texte et stocker le résultat dans une nouvelle colonne
tqdm.pandas(desc="Détection de la langue")
df['lang'] = df['text'].progress_apply(lambda x: detect_lang(str(x)))

# Afficher la distribution des langues détectées
lang_counts = df['lang'].value_counts(dropna=True)
print("Distribution des langues détectées :")
print(lang_counts)

# Identifier la langue la plus fréquente (en ignorant les éventuels None)
most_common_lang = lang_counts.idxmax()
print("\nLa langue la plus fréquente dans le dataset est :", most_common_lang)

# Initialiser le traducteur
translator = Translator()

# Fonction pour traduire un texte vers la langue cible
def translate_text(text, target_lang):
    try:
        # Si le texte est vide ou non de type string, on le retourne tel quel
        if not text or not isinstance(text, str):
            return text
        # Traduction du texte
        result = translator.translate(text, dest=target_lang)
        return result.text
    except Exception as e:
        # En cas d'erreur lors de la traduction, afficher un message et renvoyer le texte original
        print("Erreur lors de la traduction :", e)
        return text

# Fonction de traitement d'une ligne : si la langue détectée n'est pas la langue cible,
# on traduit le texte, sinon on le laisse inchangé.
def process_row(row):
    if row['lang'] != most_common_lang and row['lang'] is not None:
        return translate_text(row['text'], most_common_lang)
    else:
        return row['text']

# Traduire les textes qui ne sont pas dans la langue la plus fréquente
tqdm.pandas(desc="Traduction des textes")
df['translated_text'] = df.progress_apply(process_row, axis=1)

# Afficher les premières lignes avec le texte original, la langue détectée et le texte traduit
print("\nExemple de lignes traitées :")
print(df[['text', 'lang', 'translated_text']].head())


Détection de la langue: 100%|██████████| 19579/19579 [00:39<00:00, 490.97it/s]


Distribution des langues détectées :
lang
en    19429
fr       31
nl       22
af       13
cy       11
da       11
it       11
no        8
de        8
so        7
ca        5
pt        5
tl        4
sv        3
es        3
et        3
tr        3
id        2
Name: count, dtype: int64

La langue la plus fréquente dans le dataset est : en


Traduction des textes: 100%|██████████| 19579/19579 [01:04<00:00, 301.75it/s]


Exemple de lignes traitées :
                                                text lang  \
0  This process, however, afforded me no means of...   en   
1  It never once occurred to me that the fumbling...   en   
2  In his left hand was a gold snuff box, from wh...   en   
3  How lovely is spring As we looked from Windsor...   en   
4  Finding nothing else, not even gold, the Super...   en   

                                     translated_text  
0  This process, however, afforded me no means of...  
1  It never once occurred to me that the fumbling...  
2  In his left hand was a gold snuff box, from wh...  
3  How lovely is spring As we looked from Windsor...  
4  Finding nothing else, not even gold, the Super...  


In [33]:
print("\nExemple de lignes traduites :")
print(df.loc[df['lang'] != most_common_lang, ['text', 'translated_text']].head())



Exemple de lignes traduites :
                                                  text  \
189  But these absurdities I must not pause to detail.   
229                        Leave me; I am inexorable."   
253  Xh, pxh, pxh, Jxhn, dxn't dx sx Yxu've gxt tx ...   
294       There was Ferdinand Fitz Fossillus Feltspar.   
310  There had been nothing like order or arrangement.   

                                       translated_text  
189  But these absurdities I must not pause to detail.  
229                        Leave me; I am inexorable."  
253  Xh, pxh, pxh, Jxhn, dxhn, dxn't dx Yexu gx x g...  
294       There was Ferdinand Fitz Fossillus Feltspar.  
310  There had been nothing like order or arrangement.  
